# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("train_data_0817.csv")
test_data = pd.read_csv("test_data_0817.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## 모델링

### 모델 정의

In [10]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

### 모델 학습

Dam 모델

In [11]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=22,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.9449544624225188,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Dam = train_and_evaluate_model(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=439
    , learning_rate=0.30985993372769294
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.1781818181818182
---
Confusion Matrix:
[[7601   43]
 [ 409   49]]
---
Accuracy: 0.9442113058504074
Precision: 0.532608695652174
Recall: 0.10698689956331878




AutoClave 모델

In [12]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=34,
    min_samples_leaf=5,
    max_features=0.7473309094470472,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_AutoClave = train_and_evaluate_model(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=570
    , learning_rate=0.2040105705276999
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.1627006087437742
---
Confusion Matrix:
[[4782 2862]
 [ 164  294]]
---
Accuracy: 0.6265119723525056
Precision: 0.09315589353612168
Recall: 0.6419213973799127




Fill1 모델

In [13]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=14,
    min_samples_split=33,
    min_samples_leaf=8,
    max_features=0.7113128413756866,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Fill1 = train_and_evaluate_model(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=913
    , learning_rate=0.055237331816147595
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.17503217503217502
---
Confusion Matrix:
[[7393  251]
 [ 390   68]]
---
Accuracy: 0.9208837324117501
Precision: 0.21316614420062696
Recall: 0.14847161572052403




Fill2 모델

In [14]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=13,
    min_samples_leaf=8,
    max_features=0.6266118401157937,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_Fill2 = train_and_evaluate_model(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=293
    , learning_rate=0.620377973483163
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.12788461538461537
---
Confusion Matrix:
[[2261 5383]
 [  59  399]]
---
Accuracy: 0.3283139965440632
Precision: 0.06900726392251816
Recall: 0.87117903930131




전체공정 모델

In [15]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=28,
    min_samples_leaf=7,
    max_features=0.7331591188366589,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

train_model_All = train_and_evaluate_model(
    'ada', train_data
    , estimator=base_estimator
    , n_estimators=677
    , learning_rate=0.6713565955468803
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data 데이터로 학습한 결과:
F1 Score: 0.11603131818765242
---
Confusion Matrix:
[[ 763 6881]
 [   6  452]]
---
Accuracy: 0.14996297210565293
Precision: 0.06163916541660985
Recall: 0.9868995633187773




soft voting

In [16]:
# VotingClassifier 사용 
estimators = [
    ('dam', train_model_Dam)
    , ('autoclave', train_model_AutoClave)
    , ('fill1', train_model_Fill1)
    , ('fill2', train_model_Fill2)
    , ('all', train_model_All)
]

# VotingClassifier 학습 및 평가
# voting_clf_hard = voting_function(train_data, estimators, voting='hard')
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.3)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this wa

Voting Classifier로 학습한 결과:
F1 Score: 0.17418351477449456
---
Confusion Matrix:
[[7515  129]
 [ 402   56]]
---
Accuracy: 0.9344606270056776
Precision: 0.3027027027027027
Recall: 0.1222707423580786




In [17]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.24)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this wa

Voting Classifier로 학습한 결과:
F1 Score: 0.18114143920595532
---
Confusion Matrix:
[[7369  275]
 [ 385   73]]
---
Accuracy: 0.9185386324364354
Precision: 0.20977011494252873
Recall: 0.15938864628820962




In [18]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.26)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this wa

Voting Classifier로 학습한 결과:
F1 Score: 0.18915159944367177
---
Confusion Matrix:
[[7451  193]
 [ 390   68]]
---
Accuracy: 0.9280424586521846
Precision: 0.26053639846743293
Recall: 0.14847161572052403




In [19]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.28)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this wa

Voting Classifier로 학습한 결과:
F1 Score: 0.19089574155653452
---
Confusion Matrix:
[[7486  158]
 [ 393   65]]
---
Accuracy: 0.9319921007158726
Precision: 0.2914798206278027
Recall: 0.14192139737991266




In [20]:
# VotingClassifier 학습 및 평가
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.32)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this wa

Voting Classifier로 학습한 결과:
F1 Score: 0.15815485996705106
---
Confusion Matrix:
[[7543  101]
 [ 410   48]]
---
Accuracy: 0.9369291532954825
Precision: 0.3221476510067114
Recall: 0.10480349344978165




---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [21]:
RANDOM_STATE = 42

base_estimator = DecisionTreeClassifier(
    max_depth=22,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.9449544624225188,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

model_Dam = fit_all_train_data_function(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=439
    , learning_rate=0.30985993372769294
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=34,
    min_samples_leaf=5,
    max_features=0.7473309094470472,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=570
    , learning_rate=0.2040105705276999
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=14,
    min_samples_split=33,
    min_samples_leaf=8,
    max_features=0.7113128413756866,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=913
    , learning_rate=0.055237331816147595
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=13,
    min_samples_leaf=8,
    max_features=0.6266118401157937,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=293
    , learning_rate=0.620377973483163
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=28,
    min_samples_leaf=7,
    max_features=0.7331591188366589,
    # n_estimators=517,
    random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'ada', train_data
    , estimator=base_estimator
    , n_estimators=677
    , learning_rate=0.6713565955468803
    , random_state=RANDOM_STATE
)

/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_dam 데이터로 학습 완료


/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_autoclave 데이터로 학습 완료


/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_fill1 데이터로 학습 완료


/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data_fill2 데이터로 학습 완료


/home/dinoboy22/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ada 모델이 train_data 데이터로 학습 완료


---

In [25]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

In [30]:
x_test_dam.fillna(0, inplace=True)
x_test_autoclave.fillna(0, inplace=True)
x_test_fill1.fillna(0, inplace=True)
x_test_fill2.fillna(0, inplace=True)
x_test_all.fillna(0, inplace=True)

Voting(hard)

4개의 모델중 다수의 모델이 판단한 값을 타겟값으로 가져감

In [31]:
'''
# 예측 리스트 (하드 보팅용)
preds = [
    model_Dam.predict(x_test_dam)
    , model_AutoClave.predict(x_test_autoclave)
    , model_Fill1.predict(x_test_fill1)
    , model_Fill2.predict(x_test_fill2)
    # , model_All.predict(x_test_all)
]

# 하드 보팅 결과
final_predictions = voting(preds, method='hard')
print(sum(final_predictions))
'''

"\n# 예측 리스트 (하드 보팅용)\npreds = [\n    model_Dam.predict(x_test_dam)\n    , model_AutoClave.predict(x_test_autoclave)\n    , model_Fill1.predict(x_test_fill1)\n    , model_Fill2.predict(x_test_fill2)\n    # , model_All.predict(x_test_all)\n]\n\n# 하드 보팅 결과\nfinal_predictions = voting(preds, method='hard')\nprint(sum(final_predictions))\n"

Voting(soft)

4개의 모델의 확률의 합에 대한 평균이 일정 수치(스레스홀드) 값을 넘으면 AbNormal 값을 가짐  

In [32]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

1600


=== Message ===

작성하신 답안 제출이 완료되었습니다.

Public Score : 0.19908814

## 4. 제출하기


### 제출 파일 작성


In [33]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [34]:
df_sub['target'].value_counts()

target
Normal      15761
AbNormal     1600
Name: count, dtype: int64

In [35]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,AbNormal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal


.